In [35]:
using DelimitedFiles
using Plots
using Optim
using ImageCore
using MLDatasets
using BenchmarkTools

In [36]:
@time MNIST.convert2image(MNIST.traintensor(1))

  0.000791 seconds (335 allocations: 21.719 KiB)


In [37]:
#start
@time A = MNIST.traintensor(1:60000)

  0.817985 seconds (327 allocations: 44.881 MiB)


28×28×60000 reinterpret(N0f8, ::Array{UInt8,3}):
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0    0.0    …  0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.216  0.533  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0    …  0.0    0.675  0.992  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.071  0.886  0.992  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.671  0.992  0.992  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.118     0.859  0.992  0.831  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.141     0.992  0.992  0.529  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.369  …  0.992  0.992  0.518  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.604     0.992  0.957  0.063  0.0  0.0  0.0
 0.

In [38]:
y = MNIST.trainlabels(1:60000)

60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 3
 5
 3
 ⋮
 7
 8
 9
 2
 9
 5
 1
 8
 3
 5
 6
 8

In [39]:
N,D,P = size(A)
X = zeros(P,N*D)
for i = 1:P
    X[i,:] = A[:,:,i][:]
end
# X = [ones(P) X]
X

60000×784 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [40]:
y = y .+1
k=unique(y)

10-element Array{Int64,1}:
  6
  1
  5
  2
 10
  3
  4
  7
  8
  9

In [41]:
(N, D) = size(X)

(60000, 784)

In [42]:
δ = zeros(N,length(k))
for i = 1:length(k)
    δ[:,i] = (y .== i)  
end
δ

60000×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  

In [43]:
function loss(X,y,θ)::Float64
    u = sum(θ[y,:] .* X, dims=2)
    v = sum(exp.(θ*X'),dims=1)
    J = -sum(u)+sum(log.(v))
    J = J/N + 0.
end

loss (generic function with 1 method)

In [44]:
function gradient(X, δ, θ)::Array{Float64}
    u=exp.(θ*X')
    z=sum(u,dims=1)
#     P = exp.(X*θ') ./ z'
    P = u' ./ z'
    (P.-δ)'*X/N
end

gradient (generic function with 1 method)

In [45]:
function bgd(X,δ,α=0.01,λ=0.1, maxIterations=100)
    (N, D) = size(X)
    θ = zeros(length(k),D)
    θs = []
    push!(θs, θ)
    for i = 1:maxIterations
        ∇ = gradient(X,δ,θ)
        θ = θ - α*(∇)
#         θ = θ-α*(∇+λ*[zeros(length(k)) θ[:,2:end]])
        push!(θs, θ)
    end
    θs
end

bgd (generic function with 4 methods)

In [46]:
θ_0 = zeros(D)
T = 1000
λ=0.01
α=0.01
@time θs = bgd(X,δ,α,λ,T)

 73.888455 seconds (140.05 k allocations: 18.568 GiB, 1.10% gc time)


1001-element Array{Any,1}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 

In [47]:
θs[end,end]

10×784 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  -2.68366e-5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -2.15759e-6  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -4.3943e-6   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -3.4332e-6   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -9.16547e-6  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  -4.2298e-6   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -4.39499e-6  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      6.39608e-5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -3.91922e-6  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -5.4297e-6   0.0  0.0  0.0  0.0

In [ ]:
Js = map(θ -> loss(X, y, θ), θs)
# loss(X, y, θs[end,end])

In [ ]:
plot(1:T+1, Js,label = "loss", xlabel = "Itr",ylabel = "loss")

In [ ]:
function classify(X, θ)::Array{Int}
    scores = θ*X'
    indexs =argmax(scores, dims=1)[:]
    map(i -> (indexs[i])[1],1:size(X,1))
end
# prediction = map(i->classify(X[i,:], θ_best_lbfgs), 1:N)
prediction = classify(X, θs[end,end])
accuracy = sum(y .== prediction)/N

In [ ]:
θ_0 = zeros(length(k),D)

In [ ]:
λ = 0.01
function cost(θ::Array{Float64,2})::Float64
    u = sum(θ[y,:] .* X, dims=2)
    v = sum(exp.(θ*X'),dims=1)
    J = -sum(u)+sum(log.(v))
#     J = J/N + 0.
    J = J/N + λ*sum(θ .*θ)/2
end

In [ ]:
cost(θ_0)

In [ ]:
function grad!(∇::Array{Float64,2}, θ::Array{Float64,2})
    u=exp.(θ*X')
    z=sum(u,dims=1)
    P = u' ./ z'
    temp = (P-δ)'*X/N
    n,d = size(temp)
    for i=1:n
#         ∇[i,:]=temp[i,:]
        ∇[i,:]=temp[i,:] .+ λ*θ[i,:]
    end
end

In [ ]:
@time result_lbfgs = optimize(cost,grad!,θ_0,LBFGS())

In [ ]:
θ_best_lbfgs=Optim.minimizer(result_lbfgs)

In [ ]:
Optim.minimum(result_lbfgs)

In [ ]:
function classify(X, θ)::Array{Int}
    scores = θ*X'
    indexs =argmax(scores, dims=1)[:]
    map(i -> (indexs[i])[1],1:size(X,1))
end

In [ ]:
prediction = classify(X, θ_best_lbfgs)
accuracy = sum(y .== prediction)/N

In [ ]:
B=MNIST.testtensor(1:10000)

In [ ]:
N̂,D̂,P̂ = size(B)
X̂ = zeros(P̂,N̂*D̂)
for i = 1:P̂
    X̂[i,:] = B[:,:,i][:]
end
# X̂ = [ones(P̂) X̂]

In [ ]:
ŷ = MNIST.testlabels(1:10000)
ŷ = ŷ .+1

In [ ]:
prediction = classify(X̂, θ_best_lbfgs)
accuracy = sum(ŷ .== prediction)/P̂